### Group Members

**Ojo, Oluwaseun** | **Sholola, Oluwafunmiwo Judah**

### Context of Dataset and Goal Overview

For our dataset, we settled on automotive data from MarketCheck spanning 8 years of inventory in Canada. Each record indicates the most recent online activity for a particular vehicle, obtained via webcrawlers that aggregate info retrieved from a vast number (~65k) of dealer websites. Online activity is not defined, but it is reasonably safe (based on the 'price' attribute provided) to assume that the records indicate sale listings (and not leases, or rentals).

According to the dataset's [Kaggle link](https://www.kaggle.com/datasets/3ea0a6a45dbd4713a8759988845f1a58038036d84515ded58f65a2ff2bd32e00?resource=download), it was last updated a year ago, so we can reasonably expect 2021 to be the upper limit for the year of car manufacture.

This dataset comprises some expected attributes such as the price, make, mileage and manufacturing year of the car. It also includes more specific (and potentially more interesting) attributes such as the fuel type, the engine size, the (corporate) dealer name and site of activity (zip and province).

While our exact goal is not settled upon just yet, as we have not studied the data in-depth enough to hone in on a single phenom, listed below are a few possible phenomenon that are in contention to be studied by us in future iterations:

* correlation of price to the make and year of a vehicle
* correlation of vehicle fuel type to sales (or frequency of sales, more specifically)
* the ideal car (or kind of car) to buy based on the age of vehicles
* the ideal province to buy a used car within Canada taking price and mileage into consideration

That said, let us dive in!


### Exploring the Data...

Prior to importing our dataset, we quickly observe that the total number of records in the dataset and the number of unique VIN values **DO NOT** match. 

The reason for this disparity in the number of records and the VIN's is due to the fact that certain 
cars come up for sale multiple times within the collection window of the dataset (the past year), or they come multiple different dealer sites.

<img src="img\kaggle-id-vin.png" width="600" height="300">






In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('ca-dealers-used.csv')

# confirming what we observe at a glance on Kaggle
df['vin'].describe()

### Index selection

Logically, it makes sense to use either the **id** attribute (which is unique) or the **vin** attribute (which, ideally, would be unique in the dataset).

Since some VIN's pop up a few times in the dataset as a result of certain vehicles being traded multiple times, we have to decide:
* if we want to drop duplicate VIN's and simply represent each vehicle once throughout the dataframe, thus making the **vin** attribute unique and possible for use as the **index** of the dataframe; *or*
* if we want to keep multiple instances of a vehicle in the dataframe and utilize **id** as a single index, or **(id, vin)** as a MultiIndex **index** of the dataframe

We decided to drop rows with duplicate VIN's as we do not require multiple instances of a vehicle for the purposes of our analysis. Subsequently, we set **id** attribute as our index.



In [ ]:
# drop rows with duplicate VIN and set 'id' as our index
df_copy = df.copy()
df_copy = df_copy.drop_duplicates('vin')
df_copy = df_copy.reset_index(drop=True).set_index('id')

display(df_copy)

In [ ]:
# getting a basic, high-level overview
display(df_copy.info())

# general description of numerical attributes in dataset
display(df_copy.describe())

### Justification for dropping certain columns

The decision to drop these columns is moreso based on some domain knowledge and logic.

The **stock_no** attribute is unique to each dealer's online inventory categorization system, and is not of a singular format; some are purely numeric in nature, while some are alphanumeric. As such, it is not extremely useful across diverse sources (dealers).

In [ ]:
# display first 20 'stock_no' fields
display(df_copy['stock_no'].head(20))

We decided to drop the **trim** attribute because of the sheer number of unique values it contains per **make**. It could lead to feature explosion.

In [ ]:
display(df_copy.groupby('trim')['make'].describe())
display(sum(df_copy.groupby('trim')['make'].describe()['unique']))

display(df_copy.groupby('make')['trim'].describe())
display(sum(df_copy.groupby('make')['trim'].describe()['unique']))

The **engine_block** is a low-level attribute that has **3** possible values:
* H-engine type
* I-engine type
* V-engine type

It is not a particularly desirable attribute for our purposes. 

Also, unlike some other attributes such as the manufacturing **year**, which can be decoded from the **VIN** and are uniform/standard across various vehicle manufacturers, the **engine_block** code in the VIN differs across manufacturers (**make**) and even across **models** within the same make. 

As such, it rapidly devolves into an arduous task in trying to retrieve an attribute that we, once again, do not view as useful to our goal(s).

In [ ]:
display(df_copy.groupby('engine_block').count())

df_copy['engine_block'].describe()

The **street** and **zip** attributes are being dropped in favour of 2 other geo-based attributes, **city** and **state**.

#### Proceeding to drop the unwanted columns and reordering our retained columns

In [ ]:
# specify the columns and drop them
col_to_drop = ['stock_no', 'trim', 'engine_block', 'street', 'zip']
df_copy = df_copy.drop(columns=col_to_drop)

# specify an order of columns and order them accordingly
order_of_cols = ['vin', 'make', 'model', 'year','miles', 'price', 'fuel_type', 'engine_size', 'body_type',  'vehicle_type', 'transmission', 'drivetrain', 'seller_name', 'city', 'state']
df_copy = df_copy[order_of_cols]

# rename state to province (more appropriate to Canada)
df_copy.rename(columns={'state' : 'province'}, inplace=True)

df_copy


### Inclusion of an Additional (Derived) Column

We decided to derive a new column, **age**, based off the manufacturing **year**, which we use to prune off vehicles that fall outside a specific range. It is important to note that this is not a trimming off of outliers, rather we are choosing deal *only* with cars that were manufactured *more recently*.

This attribute might also come in handy in extracting useful features, such as the relationship between the manufacturing **year** of vehicles and the **frequency** (number) of sales.

In [ ]:
df_copy['age'] = df_copy.year.apply(lambda x: 2022-x) # creating age column to know the age of the vehicles in our dataset
print(f"Ages Before\nOldest: {df_copy['age'].max()}\nNewest: {df_copy['age'].min()}")
display(df_copy.info())

# """We will plot a line plot to see the number of vehicles by age in order to know if we should keep or drop
# rows based on the age of the vehicle"""

#Plotting the numbers of vehicles by year
df_copy['age'].value_counts().plot(kind='bar', figsize = (12, 6), title = 'Number of Vehicles by Age')
plt.xlabel('Vehicle Age')
plt.ylabel('Vehicle Count')
plt.show()

In [ ]:
"""
From our plot, we can observe that the vehicles that are newer have higher numbers than older vehicles.
We will therefore drop rows with older cars. We will work with cars between ages 1 - 15
"""

df_copy = df_copy[(df_copy['age']>= 0) & (df_copy['age'] <= 15)] # removing vehicles that are older than 20 years old
print(f"Ages After\nOldest: {df_copy['age'].max()}\nNewest: {df_copy['age'].min()}")
display(df_copy.info())

#Plotting the numbers of vehicles by year after removi g older cars
df_copy['age'].value_counts().plot(kind='bar',figsize = (12, 6), title = 'Number of Vehicles by Age')
plt.xlabel('Vehicle Age')
plt.ylabel('Vehicle Count')
plt.show()

### Contextualization and Description of Variables

Our dataset is the collection of inventory of used cars from various auto dealers across Canada. The data depicts the market activity of used cars sold by different dealers in the various provinces of Canada. The following are the variables in the dataset and their description:

**id** :    Unique identifier for each row.<br>
            *Data type: string*<br>
**vin** :   17 character long vehicle identification number. This is the value that uniquely identify each vehicle.<br> 
            *Data type: string* <br>
**price** : Price of the vehicle. This is the value that we will try to predict.<br>
            *Data type: float* <br>
**miles** : Number of miles/odometer on the vehicle. The higher the number, the more the vehicle has been driven or used.<br>
            *Data type: float* <br>
**year** :  Model year of the vehicle - Decoded from VIN. This is the year the vehicle was manufactured. <br>
            *Data type: float* <br>
**make** :  Make of the vehicle - Decoded from VIN. This is the manufacturer of the vehicle. <br>
            *Data type: string* <br>
**model** : Model of the vehicle - Decoded from VIN. It is the specific model of the vehicle. <br>
            *Data type: string* <br>
**body_type** : Body type of the vehicle - Decoded from VIN. This is the type of vehicle. For example, sedan, coupe, SUV, etc. <br>
            *Data type: string* <br>
**vehicle_type** : This is the type of vehicle. For example, car, truck, etc. <br>
            *Data type: string* <br>
**engine_size** : This is the size of the engine in liters. <br>
            *Data type: float*<br>
**fuel_type** : This is the type of fuel the vehicle uses. For example, gasoline, diesel, etc. <br>
            *Data type: string* <br>
**transmission** : Transmission type of the vehicle. The transmission type is either automatic or manual. <br>
            *Data type: string* <br>
**drivetrain** : Drivetrain type of the vehicle. The drivetrain type is either 4WD, FWD, or RWD. <br>
            *Data type: string* <br>
**seller_name** : Name of the seller or dealership. <br>
            *Data type: string* <br>
**city** : City of the seller or dealership. <br>
            *Data type: string* <br>
**state** : State of the seller or dealership. <br>
            *Data type: string* <br>



### Filling in Missing Data
Two attributes we decided on, for addressing missing data are:
* **year**; *and*
* **engine_size**

For **year**, it is **not** missing from the original dataset, so we are going to artificially introduce missing values in some rows. The intention (and result) is to repoduce these values by decoding the VIN. It is a standard across all manufacturers that the **10th digit** in the VIN is a *letter* or a *number*, which corresponds to the manufacturing year of the vehicle.

<img src="img\vin-year-dict-mapping.png" width="600" height="300">



In [110]:
# function to introduce NaN into random rows under a particular column
def add_NaN_random_rows(data, attribute, fraction):
    for col in data.columns:
        if col == attribute:
            data.loc[data.sample(frac=fraction).index, col] = np.nan

# place/introduce NaN into year for a random sample of 20% of our dataset,
# effectively mimicking missing data under the 'year' attribute 
add_NaN_random_rows(df_copy, 'year', 0.2) 
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 221742 entries, b39ea795-eca9 to 479607ed-62af
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   vin           221742 non-null  object 
 1   make          221742 non-null  object 
 2   model         219143 non-null  object 
 3   year          177394 non-null  float64
 4   miles         209571 non-null  float64
 5   price         200716 non-null  float64
 6   fuel_type     178168 non-null  object 
 7   engine_size   177244 non-null  float64
 8   body_type     201738 non-null  object 
 9   vehicle_type  199542 non-null  object 
 10  transmission  198074 non-null  object 
 11  drivetrain    196942 non-null  object 
 12  seller_name   220579 non-null  object 
 13  city          218043 non-null  object 
 14  province      217984 non-null  object 
 15  age           221742 non-null  float64
dtypes: float64(5), object(11)
memory usage: 36.8+ MB


In [ ]:
"""
We are able to fill missing year from Vehicle VIN.
This function generates year of the vehicle from the VIN.
"""

def generate_year(vin):
    decoder = {'1':'2001', '2': '2002', '3': '2003', '4':'2004', '5':'2005', '6':'2006', '7':'2007', '8':'2008', '9':'2009', 'A':'2010', 'B':'2011', 
               'C':'2012', 'D':'2013', 'E':'2014', 'F':'2015', 'G':'2016', 'H':'2017', 'J':'2018', 'K':'2019', 'L':'2020', 'M':'2021', 'N':'2022'}
    vin = str(vin)
    if (len(vin) != 17) :
        return
    else:
        year_code = str(vin)[-8]
        
        for key,value in decoder.items():
            if year_code == key:
                return value      
    
df_copy['year'] = df_copy['vin'].apply(generate_year)
df_copy.sort_values(by=['year'], ascending=False)
df_copy.info()

For **engine_size**, about **18%** of values are missing from this attribute in the original dataset. The engine size can be derived from the VIN, but the encoding is unique to each manufacturer and also to the particular model of a vehicle, so using a dictionary would be expensive (memory-wise). As such, we utilize centrality measures, based on categorization fo a related column (**body_type**), to fill missing values.

In [ ]:
# engine_size per body_type category
display(df_copy.groupby('engine_size')['body_type'].describe())

# engine_size stats based on body_type of vehicles
engine_size_stats = df_copy.groupby('body_type')['engine_size'].agg({'mean', 'median', pd.Series.mode, 'std', 'min', 'max'})
engine_size_stats = engine_size_stats[['std', 'min', 'mean', 'max', 'median', 'mode']]
display(engine_size_stats)

fig, axs = plt.subplots(1, 1, figsize=(16,16))
engine_size_stats.plot.barh(ax=axs)

# all vehicles with 'Van' body_type are missing 'engine_size' attribute
# display(df_copy[df_copy['body_type'] == 'Van']['engine_size'].info())

In [ ]:
# remove rows with NaN in body_type attr; can't derive a reasonable engine_size heuristic for them
# this is really important;
df_copy.dropna(subset=['body_type'], inplace=True)

# we save a record of rows that are currently missing their engine_size (but do not have NaN as body_type (already dropped those above))
# this way, we can compare our method of filling (using measures of centrality)
# to some engine_size values derived using some model (if we do go down that route)
engine_size_missing = df_copy[df_copy['engine_size'].isna()]
display(engine_size_missing)   

# returns some centrality measure (mean or median) of observed 'engine_sizes' for a vehicle of a certain 'body_type'
def fix_engine_size(body_type) :
    # print(engine_size_stats.loc[val, 'mean'])
    return engine_size_stats.loc[body_type, 'mean']

# replace missing engine_size with a specified mea
df_copy['engine_size'].fillna(df_copy[df_copy['engine_size'].isna()]['body_type'].apply(fix_engine_size), inplace=True)

# should be left with rows (vehicles) with no centrality values for their body_type (NaN for mean of Vans for instance)
display(df_copy[df_copy['engine_size'].isna()])                         


### Handling Outliers
For outliers, we decided to inspect all our numerical attributes, besides **year** and **age**:
* **price**
* **miles**
* **engine_size**

Let's get a preview of our outliers.


In [ ]:
def plot_box(df, columns):
    fig, axs = plt.subplots(len(columns),1,figsize=(16,18))
    i = 0
    for c in columns:
        df[c].plot.box(ax=axs[i])
        i+=1

col_to_plot = ['price', 'miles','engine_size']
plot_box(df_copy, col_to_plot)

In [ ]:
#Function to remove outliers
def remove_outliers(data, cols):
    for c in cols:
        q25 = np.percentile(data[c], 25)
        q75 = np.percentile(data[c], 75)
        iqr = q75 - q25
        lower_bound = q25 - (1.5 * iqr)
        upper_bound = q75 + (1.5 * iqr)
    
    return data[(data[c] > lower_bound) & (data[c] < upper_bound)]

col_with_outliers = col_to_plot   
df_sub = remove_outliers(df_copy, col_with_outliers)
df_sub.info()

### Final Cleanup

Finally, we drop rows missing values from certain columns:
* **vin** : fundamental attribute, cannot be retrieved from others
* **miles** : cannot be retrieved from any other columns
* **model** : is obfuscated in VIN (not a 1-to-1 mapping) since it varies across manufacturers (makes)
* **body_type** : is also obfuscated in VIN (unique to each make); if missing for a particular model of a make, cannot be retrieved from other models
* **city** : even if seller_name is available, if city is missing, it is missing in all instances of seller_name (available or also missing)  
* **province** : same reason as city


In [ ]:
# get a preview of the state of the dataset prior to dropping any rows
print(f'State of the Dataset before dropping any rows...')
display(df_copy.info())

# we drop rows missing values from certain columns
df_copy.dropna(inplace=True)

# preview after dropping some rows based on above conditions
print(f'State of the Dataset after dropping some rows...')
display(df_copy.info())

### Sub-sampling to create working dataset

Once we have dropped duplicate records, filled in missing data and taken care of outliers, it's finally time to select a sub-sample of our still dirty, but *slightly* cleaned, dataset.

For this, we created a random sampler. Given the size of our initial dataset, and the size of the desired sample, it creates bins of roughly same sizes and randomly selects indices from each of these bins. These indices correspond to the indices of rows we will select from our main dataset, which we then proceed to select.

In [ ]:
# generates and returns a random sample of indices [0...n-1] of length, size_sub_df
def random_sampler(size_initial_df,  size_sub_df):
    bin_size = round(size_initial_df/size_sub_df)
    sample_indices = []
    low = 0
    high = 1

    for i in range(0, size_sub_df+1):
        sample_indices.append(np.random.randint((low*bin_size), (high*bin_size)))
        low+=1
        high+=1

    return sample_indices

# generate a list of 1500 random indices between 0 and n-1; n=len(df)
sample_indices = random_sampler(len(df_copy), 1500)
# display(sample_indices)

# use generated indices to sample initial dataframe and create and subset
df_sub = df_copy.iloc[sample_indices, :].copy()
df_sub.info()

### Data Presentation

Since our data is not temporal, we plotted each numerical attribute against its frequency. Below are the non-normalized probability distribution functions.

In [ ]:
# because our data is not temporal, nor is it reasonable to plot against our index ('id'),
# the natural progression is to plot the 

# plots non-normalized distribution
def plot_all_series(df, columns):
    fig, axs = plt.subplots(len(columns),1,figsize=(16,18))
    axs = axs.flatten()
    i = 0
    for c in columns:
        temp_ser = df.groupby(by=[c]).size()
        axs[i].scatter(x=temp_ser.index, y=temp_ser, label=columns[i])
        axs[i].legend(loc='upper center')
        i+=1

plot_all_series(df_sub, col_to_plot)

So instead, we plot normalized probability density functions of our numerical attributes. We also utilize histograms as they better describe the congregation of certain values.

In [ ]:
def plot_histograms_density(df, columns):
    fig, axs = plt.subplots(len(columns),1,figsize=(16,18))
    i = 0
    for c in columns:
        df[c].hist(ax=axs[i], density=True)
        df[c].plot.density(ax=axs[i], title=c)
        i+=1

col_to_plot = ['price', 'miles', 'age', 'engine_size']
plot_histograms_density(df_sub, col_to_plot)